### this is the sample notebook taken directly from the repo's github.

# TF CNN Classifier

To run this notebook on an another benchmark, use

```
papermill utils/tf_cnn_classifier.ipynb tf_cnn_experiments/[DATASET NAME].ipynb -p DATASET [DATASET NAME]
```

In [20]:
import os

print(os.getcwd())

/mnt/c/Users/justi/Desktop/Home/Hackathon/HackOS_1/genomic


In [21]:
# DATASET = 'demo_coding_vs_intergenomic_seqs'
VERSION = 0
BATCH_SIZE = 64
EPOCHS = 10

In [22]:
# Parameters
DATASET = "human_ocr_ensembl"


In [23]:
print(DATASET, VERSION, BATCH_SIZE, EPOCHS)

human_ocr_ensembl 0 64 10


# Data download

In [24]:
from pathlib import Path
import tensorflow as tf
import tensorflow_addons as tfa

import numpy as np
from genomic_benchmarks.loc2seq import download_dataset
from genomic_benchmarks.data_check import is_downloaded, info
from genomic_benchmarks.models.tf import vectorize_layer
from genomic_benchmarks.models.tf import get_basic_cnn_model_v0 as get_model

# if not is_downloaded(DATASET):
#     download_dataset(DATASET)

In [25]:
from genomic_benchmarks.loc2seq import download_dataset

download_dataset("human_ocr_ensembl", version=0)

TypeError: Constructor parameter should be str

In [18]:
info(DATASET)

TypeError: Constructor parameter should be str

## TF Dataset object

In [32]:
#SEQ_PATH = Path.home() / '.genomic_benchmarks' / DATASET
SEQ_PATH = Path(DATASET)
CLASSES = [x.stem for x in (SEQ_PATH/'train').iterdir() if x.is_dir()]
NUM_CLASSES = len(CLASSES)

train_dset = tf.keras.preprocessing.text_dataset_from_directory(
    SEQ_PATH / 'train',
    batch_size=BATCH_SIZE,
    class_names=CLASSES)

Found 139804 files belonging to 2 classes.


In [35]:
if NUM_CLASSES > 2:
    train_dset = train_dset.map(lambda x, y: (x, tf.one_hot(y, depth=NUM_CLASSES)))

<_BatchDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


## Text vectorization

In [36]:
vectorize_layer.adapt(train_dset.map(lambda x, y: x))
VOCAB_SIZE = len(vectorize_layer.get_vocabulary())
vectorize_layer.get_vocabulary()

['', '[UNK]', 't', 'a', 'g', 'c', 'n']

In [ ]:
def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text)-2, label

print(text)

train_ds = train_dset.map(vectorize_text)

## Model training

In [38]:
model = get_model(NUM_CLASSES, VOCAB_SIZE)

In [ ]:
history = model.fit(
    train_ds,
    epochs=EPOCHS)

Epoch 1/10
2185/2185 [==============================] - 95s 42ms/step - loss: 0.6311 - binary_accuracy: 0.6417 - f1_score: 0.4172
Epoch 2/10
2185/2185 [==============================] - 95s 43ms/step - loss: 0.5930 - binary_accuracy: 0.6822 - f1_score: 0.5713
Epoch 3/10
2185/2185 [==============================] - 96s 44ms/step - loss: 0.5762 - binary_accuracy: 0.6985 - f1_score: 0.6074
Epoch 4/10
2185/2185 [==============================] - 99s 45ms/step - loss: 0.5665 - binary_accuracy: 0.7045 - f1_score: 0.6235
Epoch 5/10
2185/2185 [==============================] - 98s 45ms/step - loss: 0.5603 - binary_accuracy: 0.7105 - f1_score: 0.6324
Epoch 6/10
2185/2185 [==============================] - 100s 46ms/step - loss: 0.5560 - binary_accuracy: 0.7145 - f1_score: 0.6397
Epoch 7/10
2185/2185 [==============================] - 101s 46ms/step - loss: 0.5522 - binary_accuracy: 0.7163 - f1_score: 0.6432
Epoch 8/10
2185/2185 [==============================] - 104s 47ms/step - loss: 0.5488 - 

## Evaluation on the test set

In [ ]:
test_dset = tf.keras.preprocessing.text_dataset_from_directory(
    SEQ_PATH / 'test',
    batch_size=BATCH_SIZE,
    class_names=CLASSES)

if NUM_CLASSES > 2:
    test_dset = test_dset.map(lambda x, y: (x, tf.one_hot(y, depth=NUM_CLASSES)))
test_ds =  test_dset.map(vectorize_text)

In [ ]:
model.evaluate(test_ds)